# Note
This needs to be refactored to work with new csv processor

In [ ]:
publishers_type = 'Think_Tank' 
Start_time = '2013' # must be 'yyyy' or 'yyyy-mm'
End_time = '2022'
feild = 'citation name'  # 'domain' or 'citation name'
input_file_name = './kpp.csv'

#### This code draws a stacked area graph that shows how many times Palestine Chronicle, Times of Israel, and Jerusalem Post have been referred by the media/journalists in the input scope over the past three years (2018, 2019, 2020) based on an imaginary sample data file produced by the postprocessor.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast

#### We could focus on a different set of sources we are interested in and a different period of times by modifying area_list and time_list defined below.

In [ ]:
# test_list_associated_publishers = ['972Mag', 'Times of Israel', 'Jerusalem Post','Haaretz','Arutz 7','Israel Hayom','Ynet']
scope_df = pd.read_csv('./citation_scope.csv')
area_list = list(scope_df.loc[scope_df['Tag: type'] == publishers_type].Name.values)
area_list = list(set(area_list))
area_list

In [ ]:
def create_month_list(start_year_month, end_year_month):
    start_year, start_month = map(lambda x: int(x), start_year_month.split("-"))
    end_year, end_month = map(lambda x: int(x), end_year_month.split("-"))
    time_list = []
    for year in range(start_year, end_year+1):
        for month in range(1, 13):
            str_month = str(month) if month > 9 else '0' + str(month)
            if (year not in [start_year, end_year]):
                time_list.append(str(year) + '-' + str_month)
            elif (year == start_year and month >= start_month):
                time_list.append(str(year) + '-' + str_month)
            elif (year == end_year and month <= end_month):
                time_list.append(str(year) + '-' + str_month)
    return time_list

if (len(Start_time) == 4):
    time_list = [str(i) for i in list(range(int(Start_time), int(End_time)+1))]
else:
    time_list = create_month_list(Start_time, End_time)

In [ ]:
def convert_list_variables(df, list_col_names):
    '''
    Convert the column values that are lists but stored as strings
    into Python list types, based on the assumption that the format of a list looks like:
    [96adf8g9200534sf91134465, 13203fs572f502d42957dsf313]
    
    Parameters
    --------
    df: pandas dataframe
    list_col_names: list
                    The list of the column names
    '''
    for index, row in df.iterrows():
        for col_name in list_col_names:
            if pd.notnull(row[col_name]):
                print([index, col_name])
                print(row[col_name][:-1].split(" "))
                df.at[index, col_name] = row[col_name][:-1].split(" ")
                

#### The time when a source was referred is recorded based on the date of publication on which the referring article/tweet was published. e.g., If a NYT article published in 2020 refers to a Times of Israel article published in 2019, the referral is counted for 2020, not 2019.

In [ ]:
def record_number_of_references_each_in_list(df, time_interval, column_name, column_value):
    '''
    Record the number of references of sources with a specific characteristic
    (e.g., sources whose 'associated publisher' is 'Times of Israel')
    
    Parameters
    --------
    df: pandas dataframe
        data read as pandas dataframe from the postprocessed .csv file
    time_interval: string
                   'month' or 'year'
    column_name: string
                 column name to focus on. e.g., 'associated publisher'
    column_value: string
                  column value for a specific source. e.g., 'Times of Israel'
                  
    Return
    --------
    date_dic: dictionary (key:time, value:the number of references)
    '''
    date_dic = {}
    
    # for each source in the area category
    for index, row in df.iterrows():
        # record the publication date of each referring article/tweet
        if row[column_name] != '[]':
            citation_names = ast.literal_eval(row[column_name])
            for citaion in citation_names:
                if citaion == column_value:
                    if time_interval == 'year':
                        # if pd.notnull(df.at[referring_id, 'date of publication']):
                        if type(row['date of publication']) is str:
                            year = row['date of publication'][:4]
                            if type(year) is str:
                                if year not in date_dic:
                                    date_dic[year] = 1
                                else:
                                    date_dic[year] += 1
                    if time_interval == 'month':
                        # if pd.notnull(df.at[referring_id, 'date of publication']):
                        year_month = row['date of publication'][:7]
                        if type(year_month) is str:
                            if year_month not in date_dic:
                                date_dic[year_month] = 1
                            else:
                                date_dic[year_month] += 1
    return date_dic

In [ ]:
def record_number_of_references_each_in_value(df, time_interval, column_name, column_value):
    '''
    Record the number of references of sources with a specific characteristic
    (e.g., sources whose 'associated publisher' is 'Times of Israel')
    
    Parameters
    --------
    df: pandas dataframe
        data read as pandas dataframe from the postprocessed .csv file
    time_interval: string
                   'month' or 'year'
    column_name: string
                 column name to focus on. e.g., 'associated publisher'
    column_value: string
                  column value for a specific source. e.g., 'Times of Israel'
                  
    Return
    --------
    date_dic: dictionary (key:time, value:the number of references)
    '''
    date_dic = {}
    
    # for each source in the area category
    for index, row in df.loc[df[column_name] == column_value].iterrows():
        # record the publication date of each referring article/tweet
            if time_interval == 'year':
                # if pd.notnull(df.at[referring_id, 'date of publication']):
                if type(row['date of publication']) is str:
                    year = row['date of publication'][:4]
                    if type(year) is str:
                        if year not in date_dic:
                            date_dic[year] = 1
                        else:
                            date_dic[year] += 1
            if time_interval == 'month':
                # if pd.notnull(df.at[referring_id, 'date of publication']):
                year_month = row['date of publication'][:7]
                if type(year_month) is str:
                    if year_month not in date_dic:
                        date_dic[year_month] = 1
                    else:
                        date_dic[year_month] += 1
    return date_dic

In [ ]:
def record_number_of_references(df, time_interval, time_list, column_name, area_list, is_list):
    '''
    Record the number of references of sources with specific characteristics
    (e.g., sources whose 'associated publisher' is 'Times of Israel',
    those whose 'associated publisher' is 'Palestine Chronicle',
    and those whose 'associated publisher' is 'Jerusalem Post')
    
    Parameters
    --------
    df: pandas dataframe
        data read as pandas dataframe from the postprocessed .csv file
    time_interval: string
                   'month' or 'year'
    column_name: string
                 column name to focus on. e.g., 'associate publisher'
    area_list: list
               a list of column value of interest
               e.g., ['Palestine Chronicle', 'Times of Israel', 'Jerusalem Post']
                  
    Return
    --------
    a pandas dataframe with time points (e.g., 2018, 2019, 2020) as indices and
    area names (e.g., 'Palestine Chronicle', 'Times of Israel', 'Jerusalem Post')
    as column labels
    '''
    data_all_area_list = []
    
    # for each area category
    for column_value in area_list:
        # date of reference for the area
        if is_list:
            data_each_area = record_number_of_references_each_in_list(df, time_interval, column_name, column_value)
        else:
            data_each_area = record_number_of_references_each_in_value(df, time_interval, column_name, column_value)
        data_each_area_list = []
        for time in time_list:
            if time in data_each_area:
                data_each_area_list.append(data_each_area[time])
            else:
                data_each_area_list.append(0)
        data_all_area_list.append(data_each_area_list)
        
    return pd.DataFrame(list(map(list, zip(*data_all_area_list))), index=time_list, columns=area_list) 

In [ ]:
# read the postprocessed data file (.csv) as a pandas dataframe
with open(input_file_name, encoding='utf-8', newline='') as f:
    df_raw = pd.read_csv(input_file_name)

# set the IDs as indices
df_raw.set_index('id', inplace=True)

# convert the list variables that are stored as strings into Python list type
list_col_names = ['referring record id', 'tags']

# convert_list_variables(df_raw, list_col_names)

#### Below is how the imaginary sample data from the postprocessor, which I made up, looks like (I left unused cells empty):

In [ ]:
df_raw.head()



In [ ]:
# area_list=['Harel, Amos']
# df = record_number_of_references(df_raw, 'year', time_list, 'citation name', area_list, True)

In [ ]:
time_type = 'year' if len(Start_time) == 4 else 'month'
is_list = True if feild == 'citation name' else False

df = record_number_of_references(df_raw, 'year', time_list, 'citation name', area_list, is_list)

#### Below is how the data behind the stacked area chart looks like:

In [ ]:
ds = pd.DataFrame(df.sum())
order = ds.sort_values(0, ascending=False)
df = df[order.index[:10]]
df

In [ ]:
df.columns

In [ ]:
plt.stackplot(time_list,
              df.transpose(),
              labels=df.columns,
              alpha=0.8)

plt.legend(bbox_to_anchor=(1.05, 1.0),loc='upper left', fontsize='small')
plt.title("Changes in middle eastern news sources cited by Twitter")
plt.ylabel("Number of References")
plt.xlabel("Year")
# plt.tight_layout()
plt.show()

#### The data file exported as .csv file below can be used to produce a stacked area graph or graphs of similar types with other libraries/languages/tools.

In [ ]:
df.to_csv('data_for_stacked_area_chart.csv')  